In [10]:
import nibabel as nib
import numpy as np
import time
import argparse
from os.path import join as pjoin

from T1mra_dataset import T1w2MraDataset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


In [11]:
num_workers = 10
batch_size = 32
num_batches = 10
data_dir = "/Users/asagilmore/src/t1-mra/data/split/test"

print("Profiling DataLoader performance"
      f" with {num_workers} workers and batch size {batch_size}"
      f" on {num_batches} batches")

test_transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.5], std=[0.5])
])

start_time = time.time()
test_dataset = T1w2MraDataset(pjoin(data_dir, "T1W"),
                              pjoin(data_dir, "MRA"),
                              test_transform)
stop_time = time.time()
elapsed_time = stop_time - start_time
print("Time to create Dataset, "
      f"preload into mem: {elapsed_time:.4f} seconds")

print(f'Length of dataset: {len(test_dataset)}')


Profiling DataLoader performance with 10 workers and batch size 32 on 10 batches


3it [00:03,  1.09s/it]

Time to create Dataset, preload into mem: 3.2862 seconds
Length of dataset: 300


In [12]:

print(f'test first')
start_time = time.time()
out = test_dataset[0]
stop_time = time.time()

elapsed_time = stop_time - start_time
print(f"Time to load first item: {elapsed_time:.4f} seconds")

print(f'test last')
start_time = time.time()
out = test_dataset[-1]
stop_time = time.time()

elapsed_time = stop_time - start_time
print(f"Time to load last item: {elapsed_time:.4f} seconds")



test first
Time to load first item: 0.0025 seconds
test last
Time to load last item: 0.0008 seconds


In [14]:

    start_time = time.time()
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size,
                                 num_workers=1, pin_memory=True)
    stop_time = time.time()
    elapsed_time = stop_time - start_time
    print("Time to create DataLoader: "
          f"{elapsed_time:.4f} seconds")

    # warm up
    for i, _ in enumerate(test_dataloader):
        if i >= 5:
            break

    start_time = time.time()
    for i, _ in enumerate(test_dataloader):
        if i >= num_batches:
            break
    end_time = time.time()

    elapsed_time = end_time - start_time
    time_per_batch = elapsed_time / num_batches
    print(f"Time per batch: {time_per_batch:.4f} seconds")
    print(f"Total time for all batches: {elapsed_time:.4f} seconds")



Time to create DataLoader: 0.0003 seconds
